In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image 
from torch.optim.lr_scheduler import StepLR , MultiStepLR 
import os
import matplotlib.pyplot as plt
%matplotlib inline
import math
import PIL.Image
import numpy as np # linear algebra
import xml.etree.ElementTree as ET # for parsing XML
import matplotlib.pyplot as plt # to show images
from PIL import Image # to read images 
from torch.nn.utils import spectral_norm


In [ ]:
# Tensor transform
# 64x64 images!

def find_mean_std():
    transform = transforms.Compose([transforms.Resize(img_size),
                                    transforms.CenterCrop(img_size),
                                    transforms.ToTensor()])
     
    imagenet_data = datasets.ImageFolder(dog_data_dir, transform=transform) 

    data_loader = torch.utils.data.DataLoader(imagenet_data,
                                              batch_size=batch_size,
                                              shuffle=False,
                                              num_workers=0) 

    mean = 0.
    std = 0.
    i =0
    for images, _ in data_loader:
        batch_samples = images.size(0)  
        images = images.view(batch_size, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        i = i +1
        if i % 1000 == 0:
            print(str(i)+"/" +str(len(data_loader.dataset)) + " mean: "+str(mean/i)+ " std: "+str(std/i))

    mean /= len(data_loader.dataset)
    std /= len(data_loader.dataset)
    return mean , std 

In [ ]:
# helper function to un-normalize and display an image

def unnormalize(img,mean,std):
    img[0] = (img[0] * std[0]) + mean[0]  # unnormalize image = ((image * std) + mean) 
    img[1] = (img[1] * std[1]) + mean[1]
    img[2] = (img[2] * std[2]) + mean[2] 
    return img
    
def imshow(img): 
    img = transforms.ToPILImage()(img) 
    plt.imshow(img) 

def display_img(images,is_unnormalize = True,pics = 20):
    # plot the images in the batch, along with the corresponding labels
    fig = plt.figure(figsize=(25, 4))
    
    x = int(math.floor(pics/10))
    for idx in np.arange(pics):
        ax = fig.add_subplot(x, pics/x, idx+1, xticks=[], yticks=[])
        if is_unnormalize:
            imshow(unnormalize(images[idx],mean,std ))
        else:
            img = images[idx].detach().cpu().numpy()
            img = np.transpose(img, (1, 2, 0))
            img = ((img + 1)*255 / (2)).astype(np.uint8)
            plt.imshow(img) 
            #imshow(images[idx]) 

def pre_process(images, _batch_size):   
    print(images[0].name())
    _new_batch_size = _batch_size
    _images = torch.FloatTensor(_batch_size, 3, img_size, img_size )  
    return _images

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
 
class DatasetDog(Dataset): 
    
    def __init__(self, file_path, transform=None,  transform_augmentations =None):
        self.ys_mapping = {} 
        counter = 0 
        for y in os.listdir('../input/annotation/Annotation/'): # list of all breeds:
            index = y.find('-')
            if index != -1:
                index += 1  
                if y not in self.ys_mapping:
                    self.ys_mapping[y[:(index-1)] ] = [y[(index):], counter,y] 
                    counter = counter + 1
          
        self.transform = transform
        self.transform_augmentations = transform_augmentations
        
        self.labels = []
        self.imgs = []
         
        for dog_name in  os.listdir('../input/all-dogs/all-dogs/'):   
            label = dog_name 
            i = label.find('_')
            if i != -1:
                i += 1 
                label = label[:(i-1)] 
                if label in self.ys_mapping:
                    label = self.ys_mapping[label] 
                else:
                    label = [label,-1,""]
            else:
                i = label.find('.') 
                if i != -1:
                    label = [label[:(i-1)],-1,""]
                else:
                    label = [label,-1,""] 
            img = Image.open('../input/all-dogs/all-dogs/' + dog_name  )  
            breed = label[2]
            i = dog_name.find('.')  
            if label != "" and i != -1 :   
                annotation = '../input/annotation/Annotation/' + breed + '/' + dog_name[:(i)]
                if  os.path.isfile(annotation):
                    tree = ET.parse(annotation)
                    root = tree.getroot()
                    objects = root.findall('object')
                    for o in objects: 
                        bndbox = o.find('bndbox') # reading bound box
                        xmin = int(bndbox.find('xmin').text)
                        ymin = int(bndbox.find('ymin').text)
                        xmax = int(bndbox.find('xmax').text)
                        ymax = int(bndbox.find('ymax').text)
                        
                        w = np.min((xmax - xmin, ymax - ymin))
                        bbox = (xmin, ymin, xmin+w, ymin+w) 
                        img = img.crop(bbox) 
                                
                        if np.mean(img) != 0:  
                            self.labels.append(label)  
                            if self.transform is not None: 
                                img = self.transform(img)
                            
                            self.imgs.append(img)
                        #break  # select one dog per image
                        
       
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):    
       
        label = self.labels[index]
        img = self.imgs[index]
        
        if self.transform_augmentations is not None: 
            img = self.transform_augmentations(img)#torchvision.transforms.functional.to_pil_image(image))
            
        #img = torch.from_numpy(np.asarray(img))  
        #image = torch.from_numpy(np.array(img)) 
        #image = image[np.newaxis, :] 
        #image = image.permute(2, 0, 1)  
            
        return (img,label )  

In [ ]:

print(os.listdir("../input")) 
dog_data_dir = "../input"
img_size = 64  

batch_size = 32

mean = (0.5, 0.5, 0.5)#(0.4799, 0.4516, 0.3921)
std  = (0.5, 0.5, 0.5)#(0.2184, 0.2136, 0.2111) 
 
transform = transforms.Compose([transforms.Resize(img_size),
                                transforms.CenterCrop(img_size)])

# Data augmentation and converting to tensors
random_transforms = [transforms.RandomRotation(degrees=5)]
transform_augmentations = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                              #transforms.RandomApply(random_transforms, p=0.3), 
                                              transforms.ToTensor(),
                                              transforms.Normalize(std  , mean)])

 

#train_data = datasets.ImageFolder(dog_data_dir, transform=transform)     
train_data = DatasetDog(dog_data_dir, transform=transform, transform_augmentations=transform_augmentations) 
  

train_loader = torch.utils.data.DataLoader(train_data,
                                           shuffle=True,
                                           batch_size=batch_size,
                                           num_workers=4)
                                            

In [ ]:
# Define tes data loader
test_dataset_loader = torch.utils.data.DataLoader(dataset=train_data,
                                                  batch_size=20,
                                                  shuffle=True)


#for batch_i, (images, labels) in enumerate(train_loader):  
images, _ = iter(test_dataset_loader).next()   
    #print('Scaled min: ', images.min())
    #print('Scaled max: ', images.max())  
    #print(batch_i)
display_img(images,False,20)

In [ ]:
class PixelwiseNorm(nn.Module):
    def __init__(self):
        super(PixelwiseNorm, self).__init__()

    def forward(self, x, alpha=1e-8):
        """
        forward pass of the module
        :param x: input activations volume
        :param alpha: small number for numerical stability
        :return: y => pixel normalized activations
        """
        y = x.pow(2.).mean(dim=1, keepdim=True).add(alpha).sqrt()  # [N1HW]
        y = x / y  # normalize the input x volume
        return y

class MinibatchStdDev(nn.Module):
    """
    Minibatch standard deviation layer for the discriminator
    """

    def __init__(self):
        """
        derived class constructor
        """
        super(MinibatchStdDev, self).__init__()

    def forward(self, x, alpha=1e-8):
        """
        forward pass of the layer
        :param x: input activation volume
        :param alpha: small number for numerical stability
        :return: y => x appended with standard deviation constant map
        """
        batch_size, _, height, width = x.shape
        # [B x C x H x W] Subtract mean over batch.
        y = x - x.mean(dim=0, keepdim=True)
        # [1 x C x H x W]  Calc standard deviation over batch
        y = torch.sqrt(y.pow(2.).mean(dim=0, keepdim=False) + alpha)

        # [1]  Take average over feature_maps and pixels.
        y = y.mean().view(1, 1, 1, 1)

        # [B x 1 x H x W]  Replicate over group and pixels.
        y = y.repeat(batch_size,1, height, width)

        # [B x C x H x W]  Append as new feature_map.
        y = torch.cat([x, y], 1)
        # return the computed values:
        return y

In [ ]:

class Generator(nn.Module):
    def __init__(self, nz, nfeats, nchannels):
        super(Generator, self).__init__()

        # input is Z, going into a convolution
        self.conv1 = spectral_norm(nn.ConvTranspose2d(nz, nfeats * 8, 4, 1, 0, bias=False))  
        # state size. (nfeats*8) x 4 x 4
        
        self.conv2 = spectral_norm(nn.ConvTranspose2d(nfeats * 8, nfeats * 8, 4, 2, 1, bias=False)) 
        # state size. (nfeats*8) x 8 x 8
        
        self.conv3 = spectral_norm(nn.ConvTranspose2d(nfeats * 8, nfeats * 4, 4, 2, 1, bias=False)) 
        # state size. (nfeats*4) x 16 x 16
        
        self.conv4 = spectral_norm(nn.ConvTranspose2d(nfeats * 4, nfeats * 2, 4, 2, 1, bias=False)) 
        # state size. (nfeats * 2) x 32 x 32
        
        self.conv5 = spectral_norm(nn.ConvTranspose2d(nfeats * 2, nfeats, 4, 2, 1, bias=False)) 
        # state size. (nfeats) x 64 x 64
        
        self.conv6 = spectral_norm(nn.ConvTranspose2d(nfeats, nchannels, 3, 1, 1, bias=False)) 
        self.pixnorm = PixelwiseNorm()
        
    def forward(self, x):
        x = F.leaky_relu(self.conv1(x)) 
        x = F.leaky_relu(self.conv2(x))
        x = self.pixnorm(x)
        x = F.leaky_relu(self.conv3(x))
        x = self.pixnorm(x)
        x = F.leaky_relu(self.conv4(x))
        x = self.pixnorm(x)
        x = F.leaky_relu(self.conv5(x))
        x = self.pixnorm(x)
        x = torch.tanh(self.conv6(x)) 
        return x


class Discriminator(nn.Module):
    def __init__(self, nchannels, nfeats,gan_method = "standard"):
        super(Discriminator, self).__init__()
        self.nfeats = nfeats
        self.gan_method = gan_method
        # input is (nchannels) x 64 x 64
        self.conv1 = nn.Conv2d(nchannels, nfeats, 4, 2, 1, bias=False) 
        # state size. (nfeats) x 32 x 32
        
        self.conv2 = spectral_norm(nn.Conv2d(nfeats, nfeats * 2, 4, 2, 1, bias=False))
        self.bn2 = nn.BatchNorm2d(nfeats * 2)
        # state size. (nfeats*2) x 16 x 16
        
        self.conv3 = spectral_norm(nn.Conv2d(nfeats * 2, nfeats * 4, 4, 2, 1, bias=False))
        self.bn3 = nn.BatchNorm2d(nfeats * 4)
        # state size. (nfeats*4) x 8 x 8
       
        self.conv4 = spectral_norm(nn.Conv2d(nfeats * 4, nfeats * 8, 4, 2, 1, bias=False))
        self.bn4 = nn.BatchNorm2d(nfeats * 8)
        # state size. (nfeats*8) x 4 x 4
         
        #self.fc1 = nn.Linear(nfeats * 8 * 4 * 4,nfeats * 4 * 4 * 4)     
        #self.fc2 = nn.Linear(nfeats * 4 * 4 * 4 ,1)    
        #self.dropout = nn.Dropout(0.25) 
        self.batch_discriminator = MinibatchStdDev()

        self.conv5 =  spectral_norm(nn.Conv2d(nfeats * 8+1, 1, 4, 1, 0, bias=False))
        # state size. 1 x 1 x 1
        
    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = F.leaky_relu(self.bn2(self.conv2(x)), 0.2)
        x = F.leaky_relu(self.bn3(self.conv3(x)), 0.2)
        x = F.leaky_relu(self.bn4(self.conv4(x)), 0.2) 
        x = self.batch_discriminator(x)  

        #x = x.view(-1, 4 * 4 * self.nfeats*8)   
        #x = F.leaky_relu(self.fc1(x)) 
        #x = self.dropout(x) 
        #x = torch.sigmoid(self.fc2(x))  
        
        if self.gan_method == "standard":  
            x = torch.sigmoid(self.conv5(x))  
        else:
            x = self.conv5(x)
            
        return x.view(-1, 1)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) 

In [ ]:
def weights_init_normal(m):
    """
    Applies initial weights to certain layers in a model .
    The weights are taken from a normal distribution 
    with mean = 0, std dev = 0.02.
    :param m: A module or layer in a network    
    """
    # classname will be something like:
    # `Conv`, `BatchNorm2d`, `Linear`, etc.
    classname = m.__class__.__name__ 
    # TODO: Apply initial weights to convolutional and linear layers
    # for every Linear layer in a model..
    if  classname.find('Linear') != -1: 
        # apply a uniform distribution to the weights and a bias=0  
        m.weight.data.normal_(0, 0.2)
        m.bias.data.fill_(0)
    elif classname.find('Conv2d') != -1 or \
         classname.find('ConvTranspose2d') != -1:
        m.weight.data.normal_(0, 0.2) # no bais in Conv2d or ConvTranspose2d

In [ ]:
def build_network(d_conv_dim, g_conv_dim, z_size,nchannels,gan_method):
    # define discriminator and generator
    
    G = Generator(z_size, g_conv_dim, nchannels).to(device)
    D = Discriminator(nchannels,d_conv_dim,gan_method).to(device)

    # initialize model weights 
    #G.apply(weights_init_normal)
    #D.apply(weights_init_normal)

    print(D)
    print()
    print(G)
    
    return D, G

In [ ]:
#if os.path.exists('./generator.pth'):  
#    netG.load_state_dict(torch.load('generator.pth'))
#if os.path.exists('./discriminator.pth'):     
#    netD.load_state_dict(torch.load('discriminator.pth'))  

In [ ]:
import torch.optim as optim 

# Define model hyperparams
d_conv_dim = 64
g_conv_dim = 64
 
nz = 256
gan_method = "standard" # "standard" , "lsgan" , "hingegan" "relativistic_standard" "average_relativistic_standard"

criterion_BCELoss           = nn.BCELoss()
criterion_BCEWithLogitsLoss = torch.nn.BCEWithLogitsLoss() 
criterion_MSELoss           = nn.MSELoss() 

netD, netG = build_network(d_conv_dim, g_conv_dim, nz,3,gan_method) 

#if os.path.exists('./discriminator.pth'):     
#    netD.load_state_dict(torch.load('discriminator.pth')) 

In [ ]:
import datetime
num_epochs = 600
step = 0  
# keep track of loss and generated, "fake" samples
samples = []
losses = []
 
print_every = 500

lr_d = 0.0002
lr_g = 0.0002

beta1 = 0.5
   
optimizerD = optim.Adam(netD.parameters(), lr=lr_d, betas=(beta1, 0.999)) 
optimizerG = optim.Adam(netG.parameters(), lr=lr_g, betas=(beta1, 0.999))

fixed_noise = torch.randn(25, nz, 1, 1, device=device)

real_label_const = 0.7
fake_label_const = 0
batch_size = train_loader.batch_size


#generate_fake_images_code = "uniform" 
  

# step_size: at how many multiples of epoch you decay
# step_size = 1, after every 1 epoch, new_lr = lr*gamma  

# gamma = decaying factor
schedulerD = StepLR(optimizerD, step_size=10, gamma=0.99)
schedulerG = StepLR(optimizerG, step_size=10, gamma=0.99)

pace = int(num_epochs/6)

# train the network
for epoch in range(num_epochs): 
     # Decay Learning Rate
    
    if(schedulerD.get_lr()[0] >= 0.00005 and schedulerG.get_lr()[0] >= 0.00005):
        schedulerD.step()
        schedulerG.step()
    
        

    # Print Learning Rate
    print('Epoch:', epoch,'LR G ', schedulerD.get_lr())
    #print('Epoch:', epoch,'LR D ', schedulerG.get_lr())
    print(datetime.datetime.now())
    for batch_i, (train_images, train_labels) in enumerate(train_loader): 
        # ============================================
        #           INIT & PRE-PROCESS 
        # ============================================
        
        # 1. Create batch 
        # Mostly necessary for the last one because if N might not be a multiple of batch_size 
        _batch_size = train_images.size(0)  
      
         # 2. Create unlabeled images 
        _images   = torch.FloatTensor(_batch_size, 3, img_size, img_size ) 
        _images.resize_as_(train_images).copy_(train_images) 
        _images   = train_images.to(device) 
       	 
        # ============================================
        #            TRAIN THE DISCRIMINATOR
        # ============================================
 
        netD.zero_grad()    
   
        # 1. Find real pred 
        d_real_pred  = netD(_images) 

        # 2. Create label
        real_label   = torch.full((_batch_size, 1), real_label_const, device=device) + np.random.uniform(-0.1, 0.1)
        fake_label   = torch.full((_batch_size, 1), fake_label_const, device=device) + np.random.uniform(0, 0.2)
  
        # 3. Find fake pred 
        # 3.1 Generate fake images   
        noise = torch.randn(_batch_size, nz, 1, 1, device=device)
        fake_images = netG(noise)  
        # 3.2  Find fake pred          
        d_fake_pred = netD(fake_images.detach()) 
        
        # 4 Compute the discriminator losses 
        if gan_method == "standard":
            criterion = torch.nn.BCELoss()  
            d_real_loss = criterion(d_real_pred,real_label)
            d_real_loss.backward() 
 
            d_fake_loss = criterion(d_fake_pred,fake_label) 
            d_fake_loss.backward()    
            d_loss = d_real_loss + d_fake_loss
            
        elif gan_method == "relativistic_standard":
            criterion = torch.nn.BCEWithLogitsLoss()
            d_loss = criterion(d_real_pred - d_fake_pred, real_label)
            d_loss.backward()  
        
        elif gan_method == "average_relativistic_standard":
            criterion = torch.nn.BCEWithLogitsLoss()
            d_loss = (criterion(d_real_pred - torch.mean(d_fake_pred), real_label) \
                     + criterion(d_fake_pred - torch.mean(d_real_pred), fake_label))/2 
            d_loss.backward()  
            
        elif  gan_method == "lsgan":  
            d_loss = ( torch.mean((d_real_pred - torch.mean(d_fake_pred) - real_label) ** 2) + \
                       torch.mean((d_fake_pred - torch.mean(d_real_pred) + real_label) ** 2))/2
            d_loss.backward() 
            
        elif gan_method == "hingegan":
            d_loss = (torch.mean(torch.nn.ReLU()(1.0 - (d_real_pred - torch.mean(d_fake_pred)))) + \
                      torch.mean(torch.nn.ReLU()(1.0 + (d_fake_pred - torch.mean(d_real_pred)))))/2
            d_loss.backward() 
     
        # 5 Step  
        optimizerD.step()
        
        
               # =========================================
        #            TRAIN THE GENERATOR
        # =========================================
        netG.zero_grad()
 
        # 2. Create label
        real_label   = torch.full((_batch_size, 1), real_label_const, device=device)  
        fake_label   = torch.full((_batch_size, 1), fake_label_const, device=device) 
  
        # 3. Find fake pred 
        g_fake_pred = netD(fake_images)
        
        # 3. Compute the generator losses 
        if gan_method == "standard":
            criterion = torch.nn.BCELoss()   
            g_loss = criterion(g_fake_pred,real_label)
            g_loss.backward() 
        
        else: 
            g_real_pred  =  netD(_images)   
            if gan_method == "relativistic_standard":
                criterion = torch.nn.BCEWithLogitsLoss()
                g_loss = criterion(g_fake_pred  - g_real_pred, real_label)
                g_loss.backward()  
        
            elif gan_method == "average_relativistic_standard":
                criterion = torch.nn.BCEWithLogitsLoss()
                g_loss = (criterion(g_real_pred - torch.mean(g_fake_pred), fake_label ) \
                         + criterion(g_fake_pred - torch.mean(g_real_pred), real_label))/2 
                g_loss.backward()  
            
            elif  gan_method == "lsgan":  
                g_loss = ( torch.mean((g_real_pred - torch.mean(g_fake_pred) + real_label) ** 2) + \
                           torch.mean((g_fake_pred - torch.mean(g_real_pred) - real_label) ** 2))/2
                g_loss.backward() 
            
            elif gan_method == "hingegan":
                g_loss = (torch.mean(torch.nn.ReLU()(1.0 + (g_real_pred - torch.mean(g_fake_pred)))) + \
                          torch.mean(torch.nn.ReLU()(1.0 - (g_fake_pred - torch.mean(g_real_pred)))))/2
                g_loss.backward() 
        
        # 4 Step   
        optimizerG.step()
        
        if step % print_every == 0: 
            # append discriminator loss and generator loss
            losses.append((d_loss.item(), g_loss.item(),)) 
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f '
                  % (epoch + 1, num_epochs, batch_i, len(train_loader),
                     d_loss.item(), g_loss.item()))   
            
        step += 1 
    
    #torch.save(netG.state_dict(), 'gan_generator.pth')
    #torch.save(netD.state_dict(), 'gan_discriminator.pth')  
    
    ## AFTER EACH EPOCH##    
    # generate and save sample, fake images
    if epoch % 25 == 0 : 
        netG.eval() # for generating samples
        noise = torch.randn(10, nz, 1, 1, device=device)
        #noise = torch.randn(batch_size, nz, 1, 1, device=device) 
        fake_images = netG(noise).to(device)  
        display_img(fake_images.to("cpu"),False,10)
        netG.train() # back to training mode

In [ ]:
fig, ax = plt.subplots()
losses = np.array(losses)
plt.plot(losses.T[0], label='Discriminator', alpha=0.5)
plt.plot(losses.T[1], label='Generator', alpha=0.5)
plt.title("Training Losses")
plt.legend()""

In [ ]:
noise = torch.randn(20, nz, 1, 1, device=device)
gen_images = netG(noise)
images = gen_images.to("cpu").clone().detach() 
#images = images.numpy().transpose(0, 2, 3, 1)
print(images.size())         
print('Scaled min: ', images.min())
print('Scaled max: ', images.max()) 
display_img(images,False)

In [ ]:
if not os.path.exists('../output_images'):
    os.mkdir('../output_images')
im_batch_size = 50
n_images=10000
total_images = 0
for i_batch in range(0, n_images, im_batch_size):
    #gen_z = torch.randn(im_batch_size, 100, 1, 1, device=device)
    noise = torch.randn(im_batch_size, nz, 1, 1, device=device)
 
    gen_images = netG(noise)    
    total_images += gen_images.size()[0] 
    images = gen_images.to("cpu").clone().detach()
    images = images.numpy().transpose(0, 2, 3, 1)
    for i_image in range(gen_images.size(0)):
        save_image(gen_images[i_image, :, :, :], os.path.join('../output_images', f'image_{i_batch+i_image:05d}.png'))

print(total_images)
import shutil
shutil.make_archive('images', 'zip', '../output_images')